In [2]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

from openinference.instrumentation import using_attributes
from openinference.instrumentation.langchain import LangChainInstrumentor

from langchain_ollama import ChatOllama


endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))

LangChainInstrumentor().instrument(tracer_provider=tracer_provider)


if __name__ == "__main__":
    with using_attributes(
        session_id="my-test-session",
        user_id="my-test-user",
        metadata={
            "test-int": 1,
            "test-str": "string",
            "test-list": [1, 2, 3],
            "test-dict": {
                "key-1": "val-1",
                "key-2": "val-2",
            },
        },
        prompt_template="Who won the soccer match in {city} on {date}",
        prompt_template_version="v1.0",
        prompt_template_variables={
            "city": "Johannesburg",
            "date": "July 11th",
        },
        tags=["tag-1", "tag-2"],
    ):
        prompt_template = "Tell me a {adjective} joke"
        prompt = PromptTemplate(input_variables=["adjective"], template=prompt_template)

        llmmodel = ChatOllama(model="llama3.2:latest", temperature=0)
        llm = LLMChain(llm=llmmodel, prompt=prompt, metadata={"category": "jokes"})
        completion = llm.predict(adjective="funny", metadata={"variant": "funny"})
        print(completion)

Attempting to instrument while already instrumented


{
    "name": "ChatOllama",
    "context": {
        "trace_id": "0x247f9107d0ea2988d06a029418887838",
        "span_id": "0x60d2eb73024737f1",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0xc1467e1f7e962f62",
    "start_time": "2025-02-05T13:36:53.519127Z",
    "end_time": "2025-02-05T13:36:54.950494Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "session.id": "my-test-session",
        "user.id": "my-test-user",
        "tag.tags": [
            "tag-1",
            "tag-2"
        ],
        "llm.prompt_template.template": "Who won the soccer match in {city} on {date}",
        "llm.prompt_template.version": "v1.0",
        "llm.prompt_template.variables": "{\"city\": \"Johannesburg\", \"date\": \"July 11th\"}",
        "input.value": "{\"messages\": [[{\"lc\": 1, \"type\": \"constructor\", \"id\": [\"langchain\", \"schema\", \"messages\", \"HumanMessage\"], \"kwargs\": {\"content\": \"Tell me a funny joke\"